# Torus one-point functions of the $O(n)$ model

In [6]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

Number of threads: 32


  Activating new project at `~`


In [7]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=sqrt(big"2")/(big"0.8"+big"0.2"*im))
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
ndiag_indices = [(r, s) for r in 1//2:1//2:40 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 0.75*precision)
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity == 1 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    parity == -1 && (fields = filter(V -> spin(V) != 0 && V.s > 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        rmax=3, show=true,
        Ps=nothing, diagchan=nothing,
        fix=nothing,
        rels=:stu
    )
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels) # rels = :stu: structure constants equal in all channels.
        if show
            Base.show(stdout, res.str_cst.s, rmax=rmax) # print only s-channel, since others are equal.
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end;

# Count dimensions of spaces of solutions

In [8]:
fixed_constants_order = Dict(
    0 => [(:s, diag_field, 1), [(:s, V, 0) for V in fields[1:end-1]]...,],
    1 => [(:s, diag_field, 1), [(:s, V, 0) for V in fields[1:end-1] if V.s >= 0]...,],
)

nb_comb_maps = Dict(
    0 => [1, 2, 3, 4, 5, 8, 9, 12, 15, 18, 21, 26, 29, 34, 39], # number of solutions
    1 => [1, 2, 3, 4, 5, 7, 8, 10, 11, 14, 17] # number of parity-even solutions
)

precisions = [40, 40, 40, 40, 40, 40, 60, 65, 75, 85, 90, 95, 100, 130]

function check_dimension(r, s; parity=0, expected_dimension=nothing, precision=nothing)
    printstyled("=====\nr = $r\n=====\n", bold=true)
    if expected_dimension === nothing
        expected_dimension = nb_comb_maps[parity][r]
    end
    if precision === nothing
        precision = precisions[r]
    end
    blocks = precompute_blocks((r, s), fields, parity=parity, precision=precisions[r])
    fixed = fixed_constants_order[parity][1:expected_dimension]
    println("With $(length(fixed)) constants fixed:")
    solve(blocks, Sig(0, 0, 0), fix=fixed, rmax=(r + 3) // 3)
    if length(fixed) > 1
        println("With $(length(fixed)-1) constant fixed:")
        solve(blocks, Sig(0, 0, 0), fix=fixed[1:end-1], rmax=(r + 3) // 3)
    end
end;

## Parity unfixed

In [40]:
@time for r in 1:5
    check_dimension(r, 0, parity=0)
end

=====
r = 1
=====
With 1 constants fixed:
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)       │ 6.2154-4.35819im   │ 1e-43     │
│ (1, 0)          │ 5.84666-10.1272im  │ 4.7e-42   │
│ (1, 1)          │ -4.14065-1.37768im │ 7e-42     │
└─────────────────┴────────────────────┴───────────┘
=====
r = 2
=====
With 2 constants fixed:
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)       │ 0.0+0.0im          │ 0         │
│ (1, 0)          │ -934.438+210.945im │ 1.1e-41   │
│ (1, 1)          │ -597.189+296.177im │ 3.7e-41   │
│ (3//2, 0)       │ 1538.45+4136.88im  │ 3e-38     │
│ (3//2, -2//3)   │ 1587.23+3847.92im  │ 3.4e-38   │
│ (3//2, 2//3) 

In [41]:
@time for r in 6:8
    check_dimension(r, 0, parity=0)
end

=====
r = 6
=====
With 8 constants fixed:
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)   │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (2, 0)          │ 4.36101e+14+1.0602e+14im   │ 1.8e-24   │
│ (2, 1//2)       │ 0                          │ 0.014     │
│ (2, -1//2)      │ 0.0+0.0im                  │ 0         │
│ (2, 1)          │ 4.68997e+14+1.33266e+14im  │ 2.7e-24   │
│ (5//2, 0)       │ -3.87464e+18+7.04656e+17im │ 3.7e-21   │
│ (5//2, -2//5)   │ -1.15419e+18+2.3137e+17

In [42]:
@time for r in 9:13
    check_dimension(r, 0, parity=0)
end

=====
r = 9
=====
With 15 constants fixed:
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)   │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (2, 0)          │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)       │ 0.0+0.0im                  │ 0         │
│ (2, -1//2)      │ 0.0+0.0im                  │ 0         │
│ (2, 1)          │ 0.0+0.0im                  │ 0         │
│ (5//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (5//2, -2//5)   │ 0.0+0.0im             

## Parity-even solutions

In [9]:
for r = 1:10
    check_dimension(r, 0, parity=1)
end

=====
r = 1
=====
With 1 constants fixed:
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)       │ 6.2154-4.35819im   │ 4.3e-43   │
│ (1, 0)          │ 5.84666-10.1272im  │ 1.7e-41   │
│ (1, 1)          │ -4.14065-1.37768im │ 3.7e-41   │
└─────────────────┴────────────────────┴───────────┘
=====
r = 2
=====
With 2 constants fixed:
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)       │ 0.0+0.0im          │ 0         │
│ (1, 0)          │ -934.438+210.945im │ 1.1e-40   │
│ (1, 1)          │ -597.189+296.177im │ 2.2e-40   │
│ (3//2, 0)       │ 1538.45+4136.88im  │ 2.7e-37   │
│ (3//2, 2//3)    │ 1587.23+3847.92im  │ 2.9e-37   │
└──────────────

| r                       | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 |
|-------------------------|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|
|number of solutions      | 1 | 1 | 2 | 3 | 4 | 5 | 8 | 9 |12 |15 | 18 | 21 | 26 | 29 | 34 |
|number of even solutions | 1 | 1 | 2 | 3 | 4 | 5 | 7 | 8 |10 |11 | 14 | 17 |    |    |    |